In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers
!pip install torch
!pip install optuna
!pip install tqdm
import os
import random
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from nltk.corpus import stopwords
import nltk

# Download stopwords if not already done


In [ ]:
import pandas as pd
import re



true_data = pd.read_csv('/kaggle/input/hindi-and-english/True.csv')
merged_data = pd.read_csv('/kaggle/input/hindi-and-english/dataset-merged.csv')
bang_data = pd.read_csv('/kaggle/input/bangla-news/balanced_bn_data.csv')
guja_data = pd.read_csv('/kaggle/input/gujarati-news/gujarati_news.csv')
mara_data = pd.read_csv('/kaggle/input/marathi-news-tf/marathi_news.csv')
telu_data = pd.read_csv('/kaggle/input/telugu/telugu_news.csv')
# Preprocess data
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+|#\w+", "", text)  # Remove mentions and hashtags
    text = re.sub(r"[^\w\s\u0900-\u097F]", "", text)  # Keep only alphanumeric and Hindi/Bengali characters
    text = re.sub(r"\s+", " ", text)  # Remove extra whitespaces
    text = text.lower()  # Convert to lowercase for English
   
    return text

true_data['feature'] = (true_data['title'].fillna("") + " " + true_data['text'].fillna("")).apply(clean_text)
merged_data['feature'] = merged_data['text'].fillna("").apply(clean_text)
bang_data['feature'] = bang_data['content'].fillna("").apply(clean_text)
guja_data['feature'] = guja_data['text'].fillna("").apply(clean_text)
mara_data['feature'] = mara_data['text'].fillna("").apply(clean_text)
telu_data['feature'] = telu_data['text'].fillna("").apply(clean_text)

# Combine datasets
data = pd.concat([true_data[['feature', 'label']], merged_data[['feature', 'label']],bang_data[['feature','label']],guja_data[['feature','label']],mara_data[['feature','label']],telu_data[['feature','label']]])
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the dataset


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to plot the data distribution
def plot_data_distribution(data):
    # Plot the total number of labels (0 and 1) in the combined dataset
    plt.figure(figsize=(12, 6))
    
    # Barplot for label distribution
    sns.countplot(x='label', data=data, palette='Set2')
    plt.title('Distribution of Labels (0 and 1)')
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.show()

    # Plot the total number of data points per dataset
    sources = ['English (True.csv)', 'Hindi (dataset-merged.csv)', 'Bangla (balanced_bn_data.csv)','Gujarati (gujarati_news.csv)','Marathi(marathi_news.csv)','Telugu(telugu_news.csv)']
    counts = [len(true_data), len(merged_data), len(bang_data), len(guja_data),len(mara_data),len(telu_data)]

    plt.figure(figsize=(12, 6))
    sns.barplot(x=sources, y=counts, palette='Set3')
    plt.title('Number of Data Points per Dataset')
    plt.xlabel('Dataset Source')
    plt.ylabel('Number of Data Points')
    plt.show()

    # Display the overall summary of combined data
    print(f"Total Data Points in Combined Dataset: {len(data)}")
    print(f"Number of Label 0: {len(data[data['label'] == 0])}")
    print(f"Number of Label 1: {len(data[data['label'] == 1])}")

# Call the function to plot
plot_data_distribution(data)


In [ ]:
def plot_individual_label_distribution(true_data, merged_data, bang_data, guja_data, mara_data, telu_data):
    # Plot label distribution for English dataset (True.csv)
    plt.figure(figsize=(12, 6))
    sns.countplot(x='label', data=true_data, palette='coolwarm')
    plt.title('Label Distribution in English Dataset (True.csv)')
    plt.xlabel('Label (0: Fake, 1: True)')
    plt.ylabel('Count')
    plt.show()

    # Plot label distribution for Hindi dataset (dataset-merged.csv)
    plt.figure(figsize=(12, 6))
    sns.countplot(x='label', data=merged_data, palette='viridis')
    plt.title('Label Distribution in Hindi Dataset (dataset-merged.csv)')
    plt.xlabel('Label (0: Fake, 1: True)')
    plt.ylabel('Count')
    plt.show()
    plt.figure(figsize=(12, 6))
    sns.countplot(x='label', data=bang_data, palette='viridis')
    plt.title('Label Distribution in Bangla Dataset (balanced_bn_data.csv)')
    plt.xlabel('Label (0: Fake, 1: True)')
    plt.ylabel('Count')
    plt.show()
    plt.figure(figsize=(12, 6))
    sns.countplot(x='label', data=guja_data, palette='viridis')
    plt.title('Label Distribution in Gujarati Dataset (gujarati_news.csv)')
    plt.xlabel('Label (0: Fake, 1: True)')
    plt.ylabel('Count')
    plt.show()
    plt.figure(figsize=(12, 6))
    sns.countplot(x='label', data=mara_data, palette='viridis')
    plt.title('Label Distribution in Marathi Dataset (marathi_news.csv)')
    plt.xlabel('Label (0: Fake, 1: True)')
    plt.ylabel('Count')
    plt.show()
    plt.figure(figsize=(12, 6))
    sns.countplot(x='label', data=telu_data, palette='viridis')
    plt.title('Label Distribution in Telugu Dataset (telugu_news.csv)')
    plt.xlabel('Label (0: Fake, 1: True)')
    plt.ylabel('Count')
    plt.show()
    
    

    # Print summary information
    print(f"English Dataset (True.csv):\nLabel 0: {len(true_data[true_data['label'] == 0])}, "
          f"Label 1: {len(true_data[true_data['label'] == 1])}\n")
    print(f"Hindi Dataset (dataset-merged.csv):\nLabel 0: {len(merged_data[merged_data['label'] == 0])}, "
          f"Label 1: {len(merged_data[merged_data['label'] == 1])}\n")
    print(f"Bangle Dataset (balanced_bn_data.csv):\nLabel 0: {len(bang_data[bang_data['label'] == 0])}, "
          f"Label 1: {len(bang_data[bang_data['label'] == 1])}")
    print(f"Gujarati Dataset (gujarati_news.csv):\nLabel 0: {len(guja_data[guja_data['label'] == 0])}, "
          f"Label 1: {len(guja_data[guja_data['label'] == 1])}")
    print(f"Marathi Dataset (marathi_news.csv):\nLabel 0: {len(mara_data[mara_data['label'] == 0])}, "
          f"Label 1: {len(mara_data[mara_data['label'] == 1])}")
    print(f"Telugu Dataset (telugu_news.csv):\nLabel 0: {len(telu_data[telu_data['label'] == 0])}, "
          f"Label 1: {len(telu_data[telu_data['label'] == 1])}")




# Call the function to generate the plots
plot_individual_label_distribution(true_data, merged_data, bang_data, guja_data, mara_data, telu_data)


In [ ]:
# Storing datasets in a dictionary for easy access
datasets = {
    'English Data': true_data,
    'Hindi Data': merged_data,
    'Bangla Data': bang_data,
    'Gujarati Data': guja_data,
    'Marathi Data': mara_data,
    'Telugu Data': telu_data
}

# Visualization of label distributions
def plot_label_distribution(datasets):
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()
    
    for i, (name, data) in enumerate(datasets.items()):
        label_counts = data['label'].value_counts()
        sns.barplot(x=label_counts.index, y=label_counts.values, ax=axes[i], palette='viridis')
        axes[i].set_title(f"{name} Label Distribution")
        axes[i].set_xlabel('Labels')
        axes[i].set_ylabel('Count')
    
    plt.tight_layout()
    plt.savefig("label_distribution.jpg", format='jpg', dpi=300)  # Save plot as JPG
    plt.show()
def plot_combined_distribution(datasets):
    combined_data = []
    for name, data in datasets.items():
        label_counts = data['label'].value_counts().reset_index()
        label_counts.columns = ['Label', 'Count']
        label_counts['Dataset'] = name
        combined_data.append(label_counts)
    
    combined_df = pd.concat(combined_data, ignore_index=True)
    plt.figure(figsize=(12, 6))
    sns.barplot(data=combined_df, x='Dataset', y='Count', hue='Label', palette='tab10')
    plt.title("Combined Label Distribution Across Datasets")
    plt.xticks(rotation=35)
    plt.ylabel('Count')
    plt.legend(title='Labels')
    plt.savefig("combined_label_distribution.jpg", format='jpg', dpi=550)  # Save plot as JPG
    plt.show()

# Pie charts for label proportions
def plot_pie_charts(datasets):
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()
    
    for i, (name, data) in enumerate(datasets.items()):
        label_counts = data['label'].value_counts()
        axes[i].pie(label_counts, labels=label_counts.index, autopct='%1.1f%%', colors=sns.color_palette('viridis', len(label_counts)))
        axes[i].set_title(f"{name} Label Proportion")
    plt.tight_layout()
    plt.savefig("label_proportions.jpg", format='jpg', dpi=300)  # Save plot as JPG
    plt.show()

# Displaying and saving visualizations
plot_label_distribution(datasets)
plot_combined_distribution(datasets)
plot_pie_charts(datasets)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

# Tokenize data
MAX_LENGTH = min(512, int(data['feature'].apply(lambda x: len(x.split())).quantile(0.9)))
tokens = tokenizer.batch_encode_plus(
    data['feature'].tolist(),
    max_length=MAX_LENGTH,
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [ ]:
labels = torch.tensor(data['label'].values)

# Create dataset
dataset = TensorDataset(tokens['input_ids'], tokens['attention_mask'], labels)

In [ ]:
def get_data_loader(indices):
    return DataLoader(TensorDataset(*[x[indices] for x in dataset.tensors]), batch_size=batch_size)

In [ ]:
class XLMRobertaClassifier(nn.Module):
    def __init__(self, model_name):
        super(XLMRobertaClassifier, self).__init__()
        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(768, 512)  # First fully connected layer
        self.fc2 = nn.Linear(512, 384)  # New layer: 512 → 384
        self.fc3 = nn.Linear(384, 2)    # Output layer: 384 → 2
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids, attention_mask):
        # Pass input through the RoBERTa model
        _, pooled_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        
        # Fully connected layers with ReLU and dropout
        x = self.dropout(self.relu(self.fc1(pooled_output)))
        x = self.dropout(self.relu(self.fc2(x)))  # New layer
        x = self.fc3(x)
        
        # Apply softmax activation for classification
        return self.softmax(x).float()  # Ensure output is float32


In [ ]:
import torch

# Check if CUDA is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model, optimizer, and scheduler
model = XLMRobertaClassifier('xlm-roberta-base').to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
criterion = nn.NLLLoss()

In [ ]:
#path = '/kaggle/input/fake-news-detection-model/pytorch/default/1/best_model_last2.pt'
#model.load_state_dict(torch.load(path))

In [ ]:
from torch.utils.data import random_split

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
def train():
    model.train()
    total_loss = 0
    num_batches = len(train_loader)
    milestones = [int(num_batches * 0.25), int(num_batches * 0.5), int(num_batches * 0.75), num_batches - 1]
    
    for step, batch in enumerate(train_loader):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        labels = labels.long()  # Ensure labels are int64
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item()
        
        # Print progress only at milestones
        if step in milestones:
            print(f"Training Progress: {((step + 1) / num_batches) * 100:.0f}% completed | Loss: {loss.item():.4f}")

    return total_loss / num_batches

def evaluate():
    model.eval()
    total_loss = 0
    preds = []  # Initialize predictions list
    true_labels = []  # Initialize true labels list
    num_batches = len(val_loader)
    milestones = [int(num_batches * 0.25), int(num_batches * 0.5), int(num_batches * 0.75), num_batches - 1]
    
    with torch.no_grad():
        for step, batch in enumerate(val_loader):
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            labels = labels.long()  # Ensure labels are int64
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

            # Print progress only at milestones
            if step in milestones:
                print(f"Evaluation Progress: {((step + 1) / num_batches) * 100:.0f}% completed | Loss: {loss.item():.4f}")
    
    accuracy = accuracy_score(true_labels, preds)
    return total_loss / num_batches, accuracy

# Training loop with progress tracking
epochs = 5
best_valid_loss = float('inf')

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    train_loss = train()  # Train the model
    valid_loss, valid_acc = evaluate()  # Evaluate accuracy after the epoch
    scheduler.step(valid_loss)  # Update learning rate based on validation loss

    print(f"Training Loss: {train_loss:.4f}")
    print(f"Validation Loss: {valid_loss:.4f}")
    print(f"Validation Accuracy: {valid_acc:.4f}")

    # Save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best_model_last3.pt')

# Load the best model
model.load_state_dict(torch.load('best_model_last3.pt'))


In [ ]:
# best_valid_loss = float('inf')
# train_losses = []
# valid_losses = []

# for epoch in range(epochs):
#     print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
#     train_loss = train()
#     print("Training done")
#     valid_loss = evaluate()
#     print("Validation done")
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'c5_new_model_weights.pt')
#     train_losses.append(train_loss)
#     valid_losses.append(valid_loss)

#     print(f'\nTraining Loss: {train_loss:.3f}')
#     print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:
def inference(test_loader):
    model.eval()
    predictions, true_labels = [], []

    for step, batch in enumerate(test_loader):
        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch

        with torch.no_grad():
            preds = model(sent_id, mask)

        preds = preds.detach().cpu().numpy()
        labels = labels.cpu().numpy()

        predictions.append(preds)
        true_labels.append(labels)

    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)

    preds_labels = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(true_labels, preds_labels)
    # precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds_labels, average='weighted')

    print(f'Accuracy: {accuracy:.4f}')
#     print(f'Precision: {precision:.4f}')
#     print(f'Recall: {recall:.4f}')
#     print(f'F1-Score: {f1:.4f}')

    return accuracy

# Run inference after training
#accuracy, precision, recall, f1 = inference(val_loader)
accuracy = inference(val_loader)

In [ ]:
accuracy = inference(train_loader)